In [1]:
import os
currdir = os.getcwd()

In [2]:
currdir

'D:\\Projects\\MLProjects'

In [3]:
from keras.preprocessing.image import load_img,img_to_array

target_image_path = os.path.join(currdir,'potrait.jpg')
style_reference_image_path = os.path.join(currdir,'transfer_style_reference.jpg')

width , height = load_img(target_image_path).size
img_height = 400
img_width = int(width*img_height/height)

Using TensorFlow backend.


In [7]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path,target_size=(img_height,img_width))
    img = img_to_array(img)
    img = np.expand_dims(img,axis = 0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:,:,0] += 103.939    ##  these 3 steps are used for the mean
    x[:,:,1] += 116.779    ##  centering of the pixel values
    x[:,:,2] += 123.68     ##  of the given image
    x = x[:,:,::-1]      # this step tranforms the image from BGR format to RGB format
    x = np.clip(x,0,255).astype('uint8')
    return x

In [8]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1,img_height,img_width,3))

input_tensor = K.concatenate([target_image,style_reference_image,combination_image] , axis = 0)

model = vgg19.VGG19(input_tensor = input_tensor,weights = 'imagenet',include_top = False)

In [6]:
def content_loss(base,combination):
    return K.sum(K.square(combination - base))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
    gram = K.dot(features,K.transpose(features))
    return gram

def style_loss(style,combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height*img_width
    return K.sum(K.square(S-C))/(4.*(channels**2)*(size**2))

def total_variation_loss(x):
    a = K.square( x[:,:img_height-1,:img_width-1,:] - x[:,1:,:img_width-1,:] )
    b = K.square( x[:,:img_height-1,:img_width-1,:] - x[:,1:,:img_height-1,:] )
    return K.sum(K.pow(a+b,1.25))

In [9]:
outputs_dict = {layer.name:layer.output for layer in model.layers}

In [10]:
outputs_dict

{'input_2': <tf.Tensor 'concat_1:0' shape=(3, 400, 400, 3) dtype=float32>,
 'block1_conv1': <tf.Tensor 'block1_conv1_1/Relu:0' shape=(3, 400, 400, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2_1/Relu:0' shape=(3, 400, 400, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool_1/MaxPool:0' shape=(3, 200, 200, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1_1/Relu:0' shape=(3, 200, 200, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2_1/Relu:0' shape=(3, 200, 200, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool_1/MaxPool:0' shape=(3, 100, 100, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1_1/Relu:0' shape=(3, 100, 100, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2_1/Relu:0' shape=(3, 100, 100, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3_1/Relu:0' shape=(3, 100, 100, 256) dtype=float32>,
 'block3_conv4': <tf.Tensor 'block3_conv4_1/Relu:0' shape=(3, 100, 100, 256) dtype=float3

In [11]:
content_layer = 'block5_conv2'
style_layers = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1'
                ]

In [12]:
total_variation_weight = 1e-4
style_weight = 1.0
content_weight = 0.025

In [17]:
loss = K.variable(0.0)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0,:,:,:]
combination_features = layer_features[2,:,:,:]
loss = loss + content_weight*content_loss(target_image_features,combination_features)

In [18]:
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1,:,:,:]
    combination_features = layer_features[2,:,:,:]
    s1 = style_loss(style_reference_features,combination_features)
    loss += (style_weight/len(style_layers))*s1

In [19]:
loss += total_variation_weight*total_variation_loss(combination_image)

In [26]:
grads = K.gradients(loss,combination_image)[0]
fetch_loss_and_grads = K.function([combination_image],[loss,grads])

class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_value = None
    def loss(self,x):
        assert self.loss_value is None
        x = x.reshape((1,img_height,img_width,3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grads_value = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grads_value = grads_value
        return self.loss_value
    def grads(self,x):
        assert self.loss_value is not None
        grads_value = np.copy(self.grads_value)
        self.loss_value = None
        self.grads_value = None
        return grads_value
evaluator = Evaluator()

In [28]:
from scipy.optimize import fmin_l_bfgs_b
from keras.preprocessing.image import save_img
import time

result_prefix = 'neural_style_result'
iterations = 100
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration ',i)
    start_time = time.time()
    x,min_val,info = fmin_l_bfgs_b(evaluator.loss,x,fprime=evaluator.grads,maxfun = 20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    save_img(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration  0
Current loss value: 13670634000.0
Image saved as neural_style_result_at_iteration_0.png
Iteration 0 completed in 11s
Start of iteration  1
Current loss value: 7092416000.0
Image saved as neural_style_result_at_iteration_1.png
Iteration 1 completed in 11s
Start of iteration  2
Current loss value: 5106887700.0
Image saved as neural_style_result_at_iteration_2.png
Iteration 2 completed in 11s
Start of iteration  3
Current loss value: 4176121300.0
Image saved as neural_style_result_at_iteration_3.png
Iteration 3 completed in 11s
Start of iteration  4
Current loss value: 3561225200.0
Image saved as neural_style_result_at_iteration_4.png
Iteration 4 completed in 11s
Start of iteration  5
Current loss value: 3156382000.0
Image saved as neural_style_result_at_iteration_5.png
Iteration 5 completed in 11s
Start of iteration  6
Current loss value: 2754625300.0
Image saved as neural_style_result_at_iteration_6.png
Iteration 6 completed in 11s
Start of iteration  7
Current los

KeyboardInterrupt: 